# Pipeline exercise

Fortunately there are computational pipelines which enable you to process many samples jointly and which make the whole workflow more user-friendly. These pipelines also help to produce a consistent, documented and therefore reproducible workflow. Here we are going to use the [SECAPR pipeline](https://github.com/AntonelliLab/seqcap_processor) on a dataset of **Ultraconserved Elements (UCEs)** that were samples for the  in South America.

![img.png](./img/topaza_distribution_sampling_map.png)

It's not clear if the existing morphological species assignments are justified and if there might be cryptic species within these morphospecies. We want to use this UCE dataset to generate a phylogeny (species tree) of these samples to define coalescent species and see if these assignments are in agreement with population genetics analyses using SNP data extracted from the UCEs.

In this tutorial you'll go through the following steps:


![](./img/secapr_workflow.png)



_________

**-1)** First you need to **update the SECAPR pipeline** by pulling the newest version from GitHub and installing it in your cluster environment. Simply execute the command below:

In [ ]:
%%bash
sh /work/projects/forbio/bin/update_secapr.sh


_________

**0)** Let's first make sure that we are connected to the correct software environment (`forbio_env`)

In [ ]:
%%bash
module load Anaconda3/5.1.0
source activate forbio_env


_______

**1)** Then copy the pipeline tutorial folder into your directory at `/work/users/USERNAME/`.


In [ ]:
%%bash
cp -r /work/projects/forbio/tutorials_tobi/pipeline/ /work/users/USERNAME/


______

**2)** Now let's run the cleaning and trimming script for all of your samples.

<div class="alert alert-block alert-info">
We are appending `2> warnings.txt` to all following SECAPR commands because the cluster is printing a lot of annoying warning messages when loading some of the SECAPR dependencies. This command will silence those warnings and print them into the file `warnings.txt`
</div>

Every command has a help function (`-h`) that shows you the available options, which I always recommend to use before runnign a function. Check out the help function of `secapr clean_reads`:

In [ ]:
%%bash
secapr clean_reads -h 2> warnings.txt

Now run the cleaning and trimming with this command. Feel free to add any flags you feel are necessary.

In [ ]:
%%bash
secapr clean_reads --input raw_reads/ --config helpfiles/adapters_info_topaza.txt --output cleaned_reads --index single --headCrop 10 --cropToLength 240 2> warnings.txt

Once it is running **INTERRUPT THIS COMMAND** using `ctrl+c`. Since it will take around 30 minutes to clean all the samples, we instead submit this as a job script. You find the job script in the `scripts` folder. Fill in the correct paths and submit it with:

In [ ]:
%%bash
nano scripts/clean_trim_secapr.sh
sbatch scripts/clean_trim_secapr.sh

**While you are waiting** check out the [PHYLUCE pipeline](https://phyluce.readthedocs.io/en/latest/), which is what you would normally use for UCE data. On the PHYLUCE page you can also find [cool tutorials](https://phyluce.readthedocs.io/en/latest/tutorial-one.html), which you might want to play around with. Here we are using the [SECAPR](https://github.com/AntonelliLab/seqcap_processor/blob/master/documentation.ipynb) pipeline because it represents the workflow from yesterday's tutorial and it gives a better handle for other, non-UCE target capture datasets.


______

**3)** You can check the quality of the cleaned reads for all samples using the `secapr quality_check` command. This will create a plot `QC_plots.pdf` with an overview of the failed and passed tests of all samples. It's okay for this exercise if the cleaned read files are not perfect, since cleaning takes so long it's a bit of a nuisance to get the settings right within the scope of this tutorial.

In [ ]:
%%bash
secapr quality_check --input cleaned_reads/ --output quality_test 2> warnings.txt


______

**4)** Now run a de novo assembly:

In [ ]:
%%bash
secapr assemble_reads --input ./cleaned_reads/ --output ./contigs_abyss --assembler abyss 2> warnings.txt

This will take several hours. Luckily you can find already assembled contigs for these samples on the cluster (`/work/projects/forbio/data/contigs_abyss/`), which you can work with. Just copy the contigs to your working directory.

In [ ]:
%%bash
cp -r /work/projects/forbio/data/contigs_abyss/ .


______

**5)** Extract the target regions:

In [ ]:
%%bash
secapr find_target_contigs --contigs contigs_abyss/ --reference helpfiles/Tetrapods-UCE-2.5Kv1.fasta --output target_contigs 2> warnings.txt

Check the output folder and have a look at the `match_table.txt` file. Ask if you don't understand the information in the `match_table.txt` file.

<div class="alert alert-block alert-info">
Reminder: Always check the help function for every command before running it by typing `secapr NAME_OF_FUNCTION -h 2> warnings.txt`.
</div>


______

**6)** Build multiple sequence alignments (MSAs) between all our samples

In [ ]:
%%bash
secapr align_sequences --sequences target_contigs/extracted_target_contigs_all_samples.fasta --output alignments/contig_alignments/ --aligner mafft --output-format fasta --no-trim --ambiguous 2> warnings.txt

If you don't want to wait, **CANCEL** and run the script `sbatch scripts/align_contigs.sh` instead which is a lot faster since it runs on parallel on 16 cores.

Go to the output folder and check how many alignments were assembled. You can use the command `ls | wc -l` which gives you the number of files in a folder.
______

**7)** Now we run the reference assembly, using our extracted contigs as reference. We'll use the following command structure, but will submit this as a job script (`script/reference_assembly.sh`), since it will take a little while:

In [ ]:
%%bash
secapr reference_assembly --reads cleaned_reads --reference_type alignment-consensus --reference alignments/contig_alignments --output remapped_reads --min_coverage 4 2> warnings.txt

If you are not entirely sure what this command does, check the [SECAPR manual](https://github.com/AntonelliLab/seqcap_processor/blob/c4e9975629544f5b8b27ba02636effe1c4900d5d/docs/notebook/reference_assembly.ipynb) for an explanation of this command.

Submit the job script with:

In [ ]:
%%bash
sbatch script/reference_assembly.sh


<div class="alert alert-block alert-warning">
What reference did we use for read mapping? Can you argue why our choice is appropiate in this case (in case you agree)? Hint: All samples are from the same genus (Topaza) and are thus relatively closely related.
</div>

Inspect some of the files using `samtools tview`.

______

**8)** You can use the `secapr locus_selection` function to find the loci that were assembled across all samples. This function will find extract the `-n` loci with the best coverage across all samples. It will only extract loci that are present in all samples. You can set the number of loci to extract (`-n`) very high, to ensure that all loci that are present in all samples will be extracted.

In [ ]:
%%bash
secapr locus_selection --input remapped_reads --output locus_selection/ --n 2500 --read_cov 3 --reference remapped_reads/reference_seqs/joined_fasta_library.fasta 2> warnings.txt


______

**9)** Now we can build alignments from these loci:

In [ ]:
%%bash
secapr align_sequences --sequences locus_selection/joined_fastas_selected_loci.fasta --output alignments/exon_intron_alignments/ --aligner mafft --output-format fasta --no-trim --ambiguous 2> warnings.txt


<div class="alert alert-block alert-warning">
What is the difference between these alignments and the ones we created in step **6)**? Can you think of any reason why these alignments might contain more loci for more samples?
</div>

______

**10)** SECAPR also has a function that enables allele phasing. This will produce two separate BAM files per sample which in turn can be summarized into two separate sequences (allele sequences) per sample and locus:

In [ ]:
%%bash
secapr phase_alleles --input remapped_reads/ --output allele_sequences --min_coverage 3


______

**11)** Finally we can use the phased BAM files to generate allele sequence alignments (MSAs) for all samples:

In [ ]:
%%bash
secapr align_sequences --sequences allele_sequences/joined_allele_fastas.fasta --output alignments/allele_alignments/ --aligner mafft --output-format fasta --no-trim --ambiguous